In [1]:
import syft as sy
import torch as th

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/mshang/Desktop/dev_env/openMined/PySyft/venv/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


In [2]:
hook = sy.TorchHook(th)
hook.local_worker.is_client_worker = False

In [3]:
@plan(sy.placeholder.FloatTensor(2,3), sy.placeholder.FloatTensor((3,5)))
def my_plan(arg_1, arg_2):
    output = arg_1 + arg_2
    return output

plan_ptr = my_plan.send(bob)

NameError: name 'plan' is not defined

In [4]:
president = sy.VirtualWorker(hook=hook, id="president")
vice_president = sy.VirtualWorker(hook=hook, id="vice_president")
secretary = sy.VirtualWorker(hook=hook, id="secretary")

In [5]:
x = president.Promise.FloatTensor((3,5))
y = vice_president.Promise.FloatTensor((3,5))

AttributeError: 'VirtualWorker' object has no attribute 'Promise'

In [6]:
x2 = x.move(secretary)
y2 = y.move(secretary)

NameError: name 'x' is not defined

In [6]:
z = x2 + y2

In [7]:
pres_output = z.move(president)
vice_pres_output = z.move(vice_president)

In [ ]:
pres_output.declare_output(0)
vice_pres_output.declare_output(0)

In [8]:
x.keep(th.ones((3,5)), keep_id="jaw23g23")
y.keep(th.ones((3,5)), keep_id="2352351")

# still blocking for both

x.keep(th.ones((3,5)), keep_id="2352351") # this execution would finish
y.keep(th.ones((3,5)), keep_id="jaw23g23") # this execution would finish

[PointerTensor | me:44509726196 -> secretary:74544206441]

### Wishlist:

1. initialization of PromiseTensor objects on remote workers
2. I want to be able to call .move() on a remote tensor, and instead of it moving the PromiseTensor, I want it to actual extend the PromiseTensor graph across workers. 
3. I want to be able to call .keep() multiple times.
4. I want to be able to call .declare_output() so that sy.Protocol() can know which Promises are each player's output.
5. Eliminate need to set "hook.local_worker.is_client_worker = False"

In [12]:
third_party_addition = sy.Protocol(president, vice_president, secretary)

In [13]:
trump = sy.VirtualWorker(hook=hook, id="Donald Trump")
pence = sy.VirtualWorker(hook=hook, id="Mike Pence")
mike = sy.VirtualWorker(hook=hook, id="Mike Pompeo")

In [15]:
{"president": (x_ptr), "vice_president": (y_ptr,)},
{"president": (pre_output), "vice_president": (vp_output)} = third_party_addition.distribute(president=trump, 
                                                                                   vice_president=pence, 
                                                                                   secretary=mike)

x_ptr.keep(th.tensor([2,3,4,5,6], keep_id="123214").send(trump))
y_ptr.keep(th.tensor([1,1,1,1,1], keep_id="123214").send(pence))

x_ptr.keep(pre_output.value("123214"), keep_id="a23523")
y_ptr.keep(vp_output.value("123214"), keep_id="a23523")

In [ ]:
pointers_to_input_promises = pygrid_node.host(third_party_addition)